# Find Days To Use for Lineup Optimization
This notebook has code that will list statistics for games days for a season of a sport. Use this to figure out which days to use as test cases for optimizing lineup generation settings.

1. Find days to run analysis on. Pick date(s) that have a desired number of games and players.

In [ ]:
from IPython import display
import os
import ipywidgets

from fantasy_py.util.notebook import DBFileDropdown, SeasonSelector

output = ipywidgets.widgets.Output()
db_dropdown = DBFileDropdown(path_to_dbs=os.environ['FANTASY_HOME'])
display.display(db_dropdown)
season_sel = SeasonSelector(db_dropdown, output_widget=output)
display.display(season_sel)
display.display(output)

In [ ]:
from sqlalchemy import func
import qgrid
from pandas import DataFrame

from fantasy_py import db
from fantasy_py import sport

os.chdir(os.environ['FANTASY_HOME'])

# approximate players to consider in lines by sport per game
EXPECTED_PLAYERS_PER_GAME = {
    sport.NHLDatabaseManager.ABBR: 40,
    sport.NBADatabaseManager.ABBR: 25,
    sport.MLBDatabaseManager.ABBR: 9,
    sport.NFLDatabaseManager.ABBR: 22
}

def show_game_details():
    if db_dropdown.value is None:
        print("Select a DB first")
        return
    
    if len(season_sel.value) == 0:
        print("select some seasons")
        return
    
    db_obj = db.get_db_obj(db_dropdown.value)

    data = {'date': [],
            'games': []}

    with db_obj.session_scoped() as session:
        for game_date, count in session.query(db.Game.date, func.count()) \
                                       .filter(db.Game.season.in_(season_sel.value)) \
                                       .group_by(db.Game.date) \
                                       .order_by(db.Game.date) \
                                       .all():
            data['date'].append(game_date)
            data['games'].append(count)


    df = DataFrame.from_dict(data)
    assert db_obj.db_manager.ABBR in EXPECTED_PLAYERS_PER_GAME, \
        "Estimate of players per game for {} has not been specified".format(db_obj.db_manager.ABBR)

    df['players'] = df['games'] * EXPECTED_PLAYERS_PER_GAME[db_obj.db_manager.ABBR]

    display.display(qgrid.QgridWidget(df=df))
    
show_game_details()